In [1]:
import numpy as np
import pandas as pd

In [2]:
def parse_stations(name):
    l_res = []
    with open(name) as f:
        n=0
        for line in f:
            l_str = line.split()
            #print(l_str)
            res = l_str[0:5]
            #print(res)
            toto = " ".join(l_str[5:]) 
            #print(toto)
            res.append(toto)
            l_res.append(res)
            n+=1
#            if n ==10:
#                break
    return l_res

In [3]:
def NYT_counties(name):
    d_res = {}
    with open(name) as f :
        header = f.readline()
        for line in f :
            l_str = line.split(",")
            # state,county, fips
            d_res.setdefault( (l_str[2], l_str[1], l_str[3]), True  )
    return d_res


In [4]:
def counties_lat_long(name):
    with open(name) as f :
        headers = f.readline().split()
        l_headers = len(headers)
        print("len of headers : ", l_headers )
        l_res = []
        for line in f :
            l_str = line.split()
            state = l_str[0]
            fips = l_str[1]
            latitude = l_str[-2]
            longitude = l_str[-1]
            l = len(l_str)
            n_words = l - l_headers + 1
            county = " ".join( l_str[3:3+n_words] )
            #d_res.setdefault(state, []).append( [ state, county, fips, latitude, longitude ])
            l_res.append( [state, county, fips, latitude, longitude ])
        return l_res


In [6]:
    l_stations = parse_stations('/home/severine/MOOCS/UDACITY/BIG_DATA/CAPSTONE_PROJECT/DATA/WEATHER/US_ghcnd_stations.txt')
    for i,station in enumerate(l_stations):
        print(station)
        if i == 10:
            break
    print( len(l_stations))
    d_station = {}
    # 0 : ID
    # 1 : latitude
    # 2 : longitude
    # 3 : elevation
    # 4 : state
    # 5 : name
    for station in l_stations :
        d_station.setdefault(station[4], []).append(station)
#        d_station[ station[4] ].append(station)
    print(len(d_station) )
    for state, l_stt in sorted(d_station.items()):
        print(state, len(l_stt))
    


['US009052008', '43.7333', '-96.6333', '482.0', 'SD', 'SIOUX FALLS (ENVIRON. CANADA)']
['US10RMHS145', '40.5268', '-105.1113', '1569.1', 'CO', 'RMHS 1.6 SSW']
['US10adam001', '40.5680', '-98.5069', '598.0', 'NE', 'JUNIATA 1.5 S']
['US10adam002', '40.5093', '-98.5493', '601.1', 'NE', 'JUNIATA 6.0 SSW']
['US10adam003', '40.4663', '-98.6537', '615.1', 'NE', 'HOLSTEIN 0.1 NW']
['US10adam004', '40.4798', '-98.4026', '570.0', 'NE', 'AYR 3.5 NE']
['US10adam006', '40.4372', '-98.5912', '601.1', 'NE', 'ROSELAND 2.8 SW']
['US10adam007', '40.5389', '-98.4713', '588.9', 'NE', 'HASTINGS 5.4 WSW']
['US10adam008', '40.4953', '-98.2973', '566.9', 'NE', 'GLENVIL 2.3 WSW']
['US10adam010', '40.5532', '-98.6297', '622.1', 'NE', 'JUNIATA 6.9 WSW']
['US10adam011', '40.4078', '-98.6161', '593.1', 'NE', 'ROSELAND 5.2 SW']
65170
52
AK 1003
AL 1012
AR 885
AZ 1534
CA 2879
CO 4310
CT 350
DC 15
DE 124
FL 1877
GA 1261
HI 752
IA 905
ID 794
IL 1889
IN 1741
KS 1994
KY 855
LA 734
MA 664
MD 692
ME 528
MI 1304
MN 1557
MO

## COVID deaths and case per county per day, from NewYork Times

In [7]:
    ################################
    #### COVID deaths and case per county per day, from NewYork Times
    nytimes_counties = NYT_counties("/home/severine/MOOCS/UDACITY/BIG_DATA/CAPSTONE_PROJECT/DATA/us-counties.txt")
    print( "number of counties from NY Times:", len(nytimes_counties))
    for i,k in enumerate (nytimes_counties.keys()):
        print("\t",k)
        if i ==10:
            break
#    for county in nytimes_counties :
#        print(county)


number of counties from NY Times: 3274
	 ('Washington', 'Snohomish', '53061')
	 ('Illinois', 'Cook', '17031')
	 ('California', 'Orange', '06059')
	 ('Arizona', 'Maricopa', '04013')
	 ('California', 'Los Angeles', '06037')
	 ('California', 'Santa Clara', '06085')
	 ('Massachusetts', 'Suffolk', '25025')
	 ('California', 'San Francisco', '06075')
	 ('Wisconsin', 'Dane', '55025')
	 ('California', 'San Diego', '06073')
	 ('Texas', 'Bexar', '48029')


## Counties fips to geographic coordinates (latitude, longitude) 


In [42]:
gazeeter_counties= counties_lat_long("/home/severine/MOOCS/UDACITY/BIG_DATA/CAPSTONE_PROJECT/DATA/2020_Gaz_counties_national.txt")
print("number of counties from Gazeeter : ", len(gazeeter_counties))
for k in gazeeter_counties[:10]:
    print("\t",k)

len of headers :  10
number of counties from Gazeeter :  3221
	 ['AL', 'Autauga County', '01001', '32.532237', '-86.64644']
	 ['AL', 'Baldwin County', '01003', '30.659218', '-87.746067']
	 ['AL', 'Barbour County', '01005', '31.870253', '-85.405104']
	 ['AL', 'Bibb County', '01007', '33.015893', '-87.127148']
	 ['AL', 'Blount County', '01009', '33.977358', '-86.56644']
	 ['AL', 'Bullock County', '01011', '32.101759', '-85.717261']
	 ['AL', 'Butler County', '01013', '31.751667', '-86.681969']
	 ['AL', 'Calhoun County', '01015', '33.770516', '-85.827909']
	 ['AL', 'Chambers County', '01017', '32.915504', '-85.394032']
	 ['AL', 'Cherokee County', '01019', '34.069515', '-85.654242']


In [74]:
class DistanceToStation :
    def __init__(self, l_stations):
        ''' init computations for latitude and longitude'''
        self.df_stations = pd.DataFrame(l_stations, columns = ["ID", "latitude_degrees", "longitude_degrees", "elevation", "stat", "name"] )
        self.df_stations["latitude_degrees"] = self.df_stations["latitude_degrees"].apply( lambda x : float(x) )
        self.df_stations["longitude_degrees"] = self.df_stations["longitude_degrees"].apply( lambda x : float(x) )
        self.df_stations["elevation"] = self.df_stations["elevation"].apply( lambda x : float(x) )

        self.df_stations["latitude"] = self.df_stations["latitude_degrees"] * np.pi / 180.
        self.df_stations["longitude"] = self.df_stations["longitude_degrees"] * np.pi / 180.
        self.df_stations["cos_latitude"] = np.cos(self.df_stations["latitude"])
           
    
    def closest_station( self, lati, longi):
        latitude = lati * np.pi / 180.
        longitude = longi * np.pi / 180.
        cos_lat = np.cos(latitude)
        print( f'latitude : {latitude}, longitude : {longitude}, cos(latitude) : {cos_lat}')
        # Haversine formula
        print("delta lat")
        self.df_stations["delta_lat_term"] = ( np.sin( (self.df_stations["latitude"] - latitude) * 0.5 ) )**2
        print("delta longitutde")
        self.df_stations["delta_long_term"] = ( np.sin( (self.df_stations["longitude"] - longitude) * 0.5) )**2
        print("a")
        self.df_stations["a"] = self.df_stations["delta_lat_term"] + self.df_stations["delta_long_term"] \
                        * cos_lat * self.df_stations["cos_latitude"]
        self.df_stations["sqrt_a"] = self.df_stations["a"].apply( lambda x : np.sqrt(x) )
        self.df_stations["sqrt_1_a"] = self.df_stations["a"].apply( lambda x : np.sqrt(1.- x) )
        self.df_stations["angle"] = np.arctan2( np.sqrt(self.df_stations["a"]), np.sqrt( 1. - self.df_stations["a"] ) )
        closest = self.df_stations["angle"].idxmin()
        print(closest)
        return self.df_stations.iloc[closest]

### for each county, find the station closest to its 'center'


In [75]:
station_distance = DistanceToStation(l_stations);

In [76]:
# gazeeter : 
# 0 :state
# 1 : county
# 2 : fips
# 3 : latitude
# 4 : longitude

# stations :
   # 0 : ID
    # 1 : latitude
    # 2 : longitude
    # 3 : elevation
    # 4 : state
    # 5 : name


In [78]:
for county in gazeeter_counties:
    closest = station_distance.closest_station( float(county[3]), float(county[4]) )
    print(county)
    print(closest)
    break

latitude : 0.5677946486891225, longitude : -1.5122656631317155, cos(latitude) : 0.8430890051204765
delta lat
delta longitutde
a
38521
['AL', 'Autauga County', '01001', '32.532237', '-86.64644']
ID                        USC00010440
latitude_degrees              32.4706
longitude_degrees              -86.68
elevation                          61
stat                               AL
name                 AUTAUGAVILLE 3 N
latitude                     0.566719
longitude                    -1.51285
cos_latitude                 0.843667
delta_lat_term            2.89319e-07
delta_long_term           8.57706e-08
a                         3.50327e-07
sqrt_a                    0.000591884
sqrt_1_a                            1
angle                     0.000591884
Name: 38521, dtype: object


In [79]:
len(l_stations)

65170

In [80]:
l_WashingtonDC_stations = [ (i,st) for i,st in enumerate(l_stations) if st[4] == 'DC']

In [81]:
average_lat = np.average([ float(st[1][1]) for st in l_WashingtonDC_stations])
average_long = np.average([ float(st[1][2]) for st in l_WashingtonDC_stations])
average_lat, average_long

(38.92026, -77.02996)

In [82]:
l_WashingtonDC_stations

[(8467,
  ['US1DCDC0001', '38.9171', '-76.9626', '17.1', 'DC', 'WASHINGTON 3.0 ENE']),
 (8468,
  ['US1DCDC0002', '38.9574', '-77.0337', '85.0', 'DC', 'WASHINGTON 3.7 NNW']),
 (8469,
  ['US1DCDC0003', '38.9377', '-77.0991', '91.1', 'DC', 'WASHINGTON 5.0 WNW']),
 (8470,
  ['US1DCDC0006', '38.8928', '-76.9834', '21.9', 'DC', 'WASHINGTON 2.0 ESE']),
 (8471,
  ['US1DCDC0007', '38.8943', '-76.9982', '25.0', 'DC', 'WASHINGTON 1.2 SE']),
 (8472,
  ['US1DCDC0008', '38.8901', '-76.9854', '27.1', 'DC', 'WASHINGTON 2.0 ESE']),
 (8473,
  ['US1DCDC0009', '38.8766', '-77.0224', '3.0', 'DC', 'WASHINGTON 2.0 SSW']),
 (8474,
  ['US1DCDC0010', '38.9773', '-77.0245', '88.1', 'DC', 'WASHINGTON 5.0 N']),
 (8475,
  ['US1DCDC0011', '38.9303', '-77.0831', '81.4', 'DC', 'WASHINGTON 4.0 WNW']),
 (8476,
  ['US1DCDC0014', '38.9581', '-77.0817', '106.4', 'DC', 'WASHINGTON 5.1 NW']),
 (8477,
  ['US1DCDC0015', '38.9203', '-77.0456', '50.6', 'DC', 'WASHINGTON 1.9 WNW']),
 (8478,
  ['US1DCDC0019', '38.8958', '-77.0370'

In [83]:
l_WashingtonDC_gazeeter = [ (i,gz) for i,gz in enumerate(gazeeter_counties) if gz[0] == 'DC']
l_WashingtonDC_gazeeter

[(320, ['DC', 'District of Columbia', '11001', '38.904247', '-77.016517'])]

In [84]:
county = gazeeter_counties[320]
county

['DC', 'District of Columbia', '11001', '38.904247', '-77.016517']

In [85]:
closest = station_distance.closest_station( float(county[3]), float(county[4]) )


latitude : 0.6790072031591263, longitude : -1.3441918000681856, cos(latitude) : 0.7781965991706662
delta lat
delta longitutde
a
8471


In [86]:
closest

ID                         US1DCDC0007
latitude_degrees               38.8943
longitude_degrees             -76.9982
elevation                           25
stat                                DC
name                 WASHINGTON 1.2 SE
latitude                      0.678834
longitude                     -1.34387
cos_latitude                  0.778306
delta_lat_term             7.53493e-09
delta_long_term            2.55507e-08
a                          2.30104e-08
sqrt_a                     0.000151692
sqrt_1_a                             1
angle                      0.000151692
Name: 8471, dtype: object

In [71]:
station_distance.df_stations.iloc[8478]

ID                       US1DCDC0019
latitude_degrees             38.8958
longitude_degrees            -77.037
elevation                        9.1
stat                              DC
name                 THE WHITE HOUSE
latitude                     0.67886
longitude                   -1.34455
cos_latitude                0.778289
delta_lat_term           5.43375e-09
delta_long_term          3.19508e-08
a                           0.605662
sqrt_a                      0.778243
sqrt_1_a                    0.627963
angle                       0.891863
Name: 8478, dtype: object

In [72]:
gazeeter_counties[320]

['DC', 'District of Columbia', '11001', '38.904247', '-77.016517']

0.09